<a href="https://colab.research.google.com/github/fortune901/nubase-project/blob/main/project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTER LES DOCUMENTS DANS LE FICHIER PYTHON
from google.colab import files
uploaded = files.upload()

In [ ]:

#importation des bibliothèques nécessaires

import pandas as pd # pour lire et manipuler les données tabulaires (fichier.csv)
from sklearn.model_selection import train_test_split # pour diviser les données en ensemble d'entrainement et de test
from sklearn.preprocessing import StandardScaler # pour normaliser les données
from tensorflow.keras.models import Sequential # ici c'est pour construire un modèle de reseau de neurone
from tensorflow.keras.layers import Dense # ici c'est pour pouvoir ajouter des couches de neurones cachees

import re

def convertir_demi_vie_en_secondes(demi_vie_texte):
    if not demi_vie_texte or demi_vie_texte.strip() == '':
        return None

    # Dictionnaire des unités vers secondes
    conversion = {
        's': 1,
        'ms': 1e-3,
        'us': 1e-6,
        'ns': 1e-9,
        'min': 60,
        'h': 3600,
        'H': 3600,
        'd': 86400,
        'y': 31536000,
        'a': 31536000,  # parfois y = a = année
    }

    # Extraction avec expression régulière
    match = re.match(r'([\d\.]+)\s*([a-zA-Zµ]+)', demi_vie_texte.strip())
    if match:
        valeur = float(match.group(1))
        unite = match.group(2).lower()  # on passe l’unité en minuscule

        for cle in conversion:
            if cle.lower() == unite:
                return valeur * conversion[cle]

    return None  # si non reconnu
#Extraction des donnees
def extraire_donnees_nubase(fichier):
  """
  Extrait les donnees nucleaires depuis le fichier nubase_4.mas20.
  """
  donnees = []
  with open(fichier, 'r', encoding='utf-8') as f:
    for ligne in f:
      if ligne.strip() == '' or ligne.startswith('#'): # cette ligne de code permet de sauter les lignes vides ou de commmentaire
        continue
      try:
        #Lectures des champs depuis les positions fixes (selon la structure de nubase2020)
        # attention : les indices peuvent varier selon les versions du fichier
        Z = ligne[0:3].strip() # Numero atomique (proton)
        N = ligne[4:8].strip() # Nombre de neutrons
        A = ligne[9:13].strip() # Nombre de masse
        element = ligne[14:17].strip() # Symbole de l'element
        Q= ligne[23:37].strip() # état fonfamentale (g) ou isomere (m)
        donnees.append({'Z': Z,
                       'N': N,
                       'A': A,
                       'Element': element,
                       #'Etat': etat,
                       'Q(KeV)': Q
        })
                        #'Demi-vie (s)': demi_vie_s, #ajout de la demi-vie en secondes
                       #'Deformation': deformation,
                       #'Shell_effect': shell_effect,
                       #'Spin_orbite': spin_orbite})
      except IndexError: # si une ligne est trop courte on l'ignore
        continue
  return donnees

# Utilisation du script
fichier = 'AME2020.dat'#'nubase_4.mas20'
donnees = extraire_donnees_nubase(fichier)
#affichage de toutes les lignes extraites
for ligne in donnees:
  print(ligne)


In [ ]:
import csv
# Enregistrement dans un fichier CSV
with open('donnees_nubase.csv', 'w', newline='', encoding='utf-8') as csvfile:
  champs = ['Z', 'N','A','Element',  'Q(KeV)']
  writer = csv.DictWriter(csvfile, fieldnames=champs)
  writer.writeheader()
  writer.writerows(donnees)

# cONVERTIR LA LISTE DE DICTIONNAIRES EN DATAFRAME
df = pd.DataFrame(donnees)
df
df.to_csv("donnees_nubase.csv", index=False)
df[['N','Z', 'Q(KeV)']]
from google.colab import files
files.download("donnees_nubase.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df = pd.read_csv('donnees_nubase.csv')
df.head()
#supprimer les colonnes qui ne sont pas utile (optionnel)
#df.drop(columns=['Colonneinutile1', 'colonneinutile2'], inplac=True)
print(df.dtypes) #Verfier les types de données

In [ ]:
# Supprimer les espaces en trop
df.columns = df.columns.str.strip()
print(df.isnull().sum()) # regarde les valeurs manquantes

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd # Ensure pandas is imported

# Cherchons les colonnes numeriques à normaliser
scaler = MinMaxScaler()
cols_a_normaliser = ['Z', 'N', 'A', 'Q(KeV)']

# Convertir les colonnes en type numérique, en gérant les erreurs
for col in cols_a_normaliser:
    # errors='coerce' remplacera les valeurs non convertibles par NaN
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Gérer les valeurs manquantes (NaN) créées par la conversion
# Option 1: Remplacer les NaNs par une valeur (ex: la moyenne, la médiane, ou 0)
# df.fillna(df.mean(), inplace=True) # Remplace par la moyenne de chaque colonne
# df.fillna(0, inplace=True)       # Remplace par 0

# Option 2: Supprimer les lignes contenant des NaNs dans ces colonnes
df.dropna(subset=cols_a_normaliser, inplace=True) # Supprime les lignes si NaN dans l'une des colonnes spécifiées

# Maintenant, appliquer le scaler sur les colonnes nettoyées
df[cols_a_normaliser] = scaler.fit_transform(df[cols_a_normaliser])

print(df.head())
print(df.describe())

In [ ]:
from sklearn.model_selection import train_test_split

X = df[['Z', 'N', 'A']]  # Ajoute d'autres colonnes si besoin
y = df['Q(KeV)']

# Exemple de séparation
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)

# choix du modèle de ML
#from sklearn.ensemble import RandomForestRegressor

#model = RandomForestRegressor(n_estimators=100, random_state=42)
#model.fit(X_train, y_train)
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', max_iter=1000)
model.fit(X_train, y_train)

# validation croisée
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, cv=7, scoring='neg_mean_squared_error')
# evaluation du modèle
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"R²: {r2}")

#prediction de Q
Q_pred = model.predict(X_test)
print(Q_pred)

In [ ]:
#evaluation du modèle de prediction
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

rmse = np.sqrt(mean_squared_error(y_test, Q_pred))
r2 = r2_score(y_test, Q_pred)

print(f"RMSE: {rmse}")
print(f"R²: {r2}")

In [ ]:
#prediction des demi-vies
def predict_half_life(Q_alpha, Z, a, b, c, d):
    # Ensure Z and Q_alpha are numpy arrays for element-wise operations
    Z = np.asarray(Z)
    Q_alpha = np.asarray(Q_alpha)

    # Handle potential division by zero or negative Q_alpha values
    # Replacing non-positive Q_alpha with a small positive number to avoid errors
    Q_alpha[Q_alpha <= 0] = 1e-9 # Replace with a small positive value

    log_T = (a * Z + b) / np.sqrt(Q_alpha) + (c * Z + d)
    T = 10**log_T  # Convertir log(T) en T
    return T

# Exemple d'utilisation
# Use the 'Z' column from the X_test DataFrame
predicted_half_lives = predict_half_life(Q_pred, X_test['Z'], a=-0.5, b=20, c=0.1, d=-5)
print(predicted_half_lives)